In [43]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import time
%matplotlib inline

In [44]:
NUM_ITERATIONS = 10
def avg(l):
    return float(sum(l))/(len(l))

In [100]:
query1_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?sensor ?room
WHERE {

    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    {?sensor bf:isPointOf ?vav }
    UNION
    {?sensor bf:isPointOf ?room }
}
"""
query1_hod = """
SELECT ?sensor ?room
WHERE {
    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    { ?sensor bf:isPointOf ?vav .
    OR
    ?sensor bf:isPointOf ?room . }
};
"""
query2_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { ?vlv_cmd rdf:type brick:Reheat_Valve_Command }
      UNION
      { ?vlv_cmd rdf:type brick:Cooling_Valve_Command }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
}
"""
query2_hod = """
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { 
      ?vlv_cmd rdf:type brick:Reheat_Valve_Command .
      OR
      ?vlv_cmd rdf:type brick:Cooling_Valve_Command .
      }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
};
"""


## Hod Benchmark

In [82]:
# Hod Configs
HOD_SERVER = "http://localhost:47808/query"

In [83]:
# run hod
#!cd hod; ./runserver.sh
!cd hod; docker kill hod ; docker rm hod; docker run -d --name hod -p47808:47808 gtfierro/hod

Error response from daemon: Cannot kill container hod: Container 95ea3267f4412a416e196ac4b341580ffedb472b0982d2368f2e4abed3e2f632 is not running
hod
ab7ae277e28a41342ea95245b04b32614bd9a30ceb09f411f985105a6990e841
docker: Error response from daemon: driver failed programming external connectivity on endpoint hod (62734627c76c23cc70707557f2214a422ad5c5af3538c41f5d99211aa936866a): Error starting userland proxy: listen tcp 0.0.0.0:47808: bind: address already in use.


In [88]:
## query 1
hod_query_1_results = []
for i in range(NUM_ITERATIONS):
    t1 = time.time()*1000
    resp = requests.post(HOD_SERVER, data=query1_hod)
    if not resp.ok:
        print resp, resp.reason
        break
    t2 = time.time()*1000
    hod_query_1_results.append(t2-t1)
    print i, '{0:.2f}'.format(t2-t1)
    time.sleep(.5)
hod_query_1_avg = avg(hod_query_1_results)
print "Took {0:.2f}ms".format(hod_query_1_avg)

0 57.15
1 27.94
2 31.81
3 63.04
4 25.47
5 22.50
6 28.26
7 39.67
8 22.36
9 27.53
Took 34.57ms


In [93]:
## query 2
hod_query_2_results = []
for i in range(NUM_ITERATIONS):
    t1 = time.time()*1000
    resp = requests.post(HOD_SERVER, data=query2_hod)
    if not resp.ok:
        print resp, resp.reason
        break
    t2 = time.time()*1000
    hod_query_2_results.append(t2-t1)
    print i, '{0:.2f}'.format(t2-t1)
    time.sleep(.5)
hod_query_2_avg = avg(hod_query_2_results)
print "Took {0:.2f}ms".format(hod_query_2_avg)

0 146.32
1 109.46
2 93.05
3 123.58
4 104.49
5 95.62
6 109.11
7 93.87
8 113.97
9 188.81
Took 117.83ms


## Fuseki Benchmark

In [96]:
# Fuseki Configs
FUSEKI_SERVER = "http://pantry.cs.berkeley.edu:3030/SodaHall/query"

In [ ]:
# TODO: on fuseki. Automatically load in brick/brickframe/berkeley ttl files in the docker entrypoint

In [ ]:
## query 1
fuseki_query_1_results = []
for i in range(NUM_ITERATIONS):
    t1 = time.time()*1000
    resp = requests.post(FUSEKI_SERVER, params={'query':query1_sparql})
    if not resp.ok:
        print resp, resp.reason
        break
    t2 = time.time()*1000
    fuseki_query_1_results.append(t2-t1)
    print i, '{0:.2f}'.format(t2-t1)
    time.sleep(.5)
fuseki_query_1_avg = avg(fuseki_query_1_results)
print "Took {0:.2f}ms".format(fuseki_query_1_avg)

In [ ]:
## query 2
fuseki_query_2_results = []
for i in range(NUM_ITERATIONS):
    t1 = time.time()*1000
    resp = requests.post(HOD_SERVER, data=query2_sparql)
    if not resp.ok:
        print resp, resp.reason
        break
    t2 = time.time()*1000
    fuseki_query_2_results.append(t2-t1)
    print i, '{0:.2f}'.format(t2-t1)
    time.sleep(.5)
fuseki_query_2_avg = avg(fuseki_query_2_results)
print "Took {0:.2f}ms".format(fuseki_query_2_avg)